Домашнее задание.
1. Попробуйте обучить нейронную сеть с применением одномерных сверток для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech
2. Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
max_words = 2000
max_len = 20
num_classes = 1

epochs = 30
batch_size = 512
print_batch_n = 100

In [3]:
df_train = pd.read_csv('twitter_data/train.csv')
df_test = pd.read_csv('twitter_data/test.csv')

In [4]:
df_train, df_val = train_test_split(df_train, test_size=0.3, shuffle=True)

In [5]:
!pip install stop-words pymorphy2

Defaulting to user installation because normal site-packages is not writeable


In [6]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [7]:
sw = set(get_stop_words("en"))

In [8]:
puncts = set(punctuation)

In [9]:
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
#     txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [10]:
from tqdm import tqdm
tqdm.pandas()

df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
df_val['tweet'] = df_val['tweet'].progress_apply(preprocess_text)
df_test['tweet'] = df_test['tweet'].progress_apply(preprocess_text)

100%|███████████████████████████████████| 17197/17197 [00:02<00:00, 6452.78it/s]


In [11]:
train_corpus = " ".join(df_train["tweet"])
train_corpus = train_corpus.lower()

In [12]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)
tokens[:5]

[nltk_data] Downloading package punkt to /home/boormistr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['user', 'one', 'awesome', 'displays', 'bbc']

In [14]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [15]:
from nltk.probability import FreqDist

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
len(tokens_filtered_top)

1999

In [16]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [17]:
import numpy as np
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return result[-maxlen:] + padding

In [18]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["tweet"]])
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["tweet"]])
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["tweet"]])

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [35]:
class LSTM(nn.Module):
    def __init__(self, vocab_size=2000, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):        
        x = self.embeddings(x)
        x = self.dropout(x)
        
        lstm_out, ht = self.lstm(x)
        
        if self.use_last:
            last_tensor = lstm_out[:,-1,:]
        else:
            last_tensor = torch.mean(lstm_out[:,:], dim=1)
        
        out = self.linear(last_tensor)
        
        return torch.sigmoid(out)

In [36]:
from torch.utils.data import DataLoader, Dataset

class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data)
        self.target = torch.from_numpy(target)
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [37]:
train_dataset = DataWrapper(x_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_val, df_val['label'].values)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [38]:
model = LSTM(max_words, 128, 128, use_last=False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [39]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [40]:
model = model.to(device)
model.train()
th = 0.5

train_loss_history = []
test_loss_history = []

for epoch in range(epochs):
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        
        loss = loss.item()
        running_items += len(labels)
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
    
    model.eval()
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
          f'Step: [{i + 1}/{len(train_loader)}]. '\
          f'Loss: {loss:.3f}, ' \
          f'Acc: {running_right / running_items:.5f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)
    
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = model(data[0].to(device))
        
        test_loss = criterion(test_outputs, test_labels.float().view(-1,1))
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
        
    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss: .3f}. \
            Test acc: {test_running_right / test_running_total: .5f}')
    
    model.train()
    
print('Training is finished!')

Epoch [1/30]. Step: [44/44]. Loss: 0.189, Acc: 0.93215. Test loss:  0.187.             Test acc:  0.94243
Epoch [2/30]. Step: [44/44]. Loss: 0.168, Acc: 0.94677. Test loss:  0.170.             Test acc:  0.95046
Epoch [3/30]. Step: [44/44]. Loss: 0.119, Acc: 0.95870. Test loss:  0.146.             Test acc:  0.95297
Epoch [4/30]. Step: [44/44]. Loss: 0.098, Acc: 0.96473. Test loss:  0.135.             Test acc:  0.95245
Epoch [5/30]. Step: [44/44]. Loss: 0.105, Acc: 0.97037. Test loss:  0.129.             Test acc:  0.95265
Epoch [6/30]. Step: [44/44]. Loss: 0.085, Acc: 0.97269. Test loss:  0.146.             Test acc:  0.95015
Epoch [7/30]. Step: [44/44]. Loss: 0.083, Acc: 0.97680. Test loss:  0.215.             Test acc:  0.94504
Epoch [8/30]. Step: [44/44]. Loss: 0.061, Acc: 0.97904. Test loss:  0.181.             Test acc:  0.94713
Epoch [9/30]. Step: [44/44]. Loss: 0.032, Acc: 0.98369. Test loss:  0.247.             Test acc:  0.95245
Epoch [10/30]. Step: [44/44]. Loss: 0.042, Acc